# SQL DATA ANALYSIS TASK

# Context
Fenix is a company that is in renewable energy. To this end, it sells solar kits to populations who are in need. Customers have the option of paying for the kit in cash or on credit. If the customer chooses to buy on credit, he can choose between one year (360 days) or two years (720). He is then asked to pay an amount per day (daily_rate), this amount depending on the duration of the loan and the type of kit he has chosen and corresponds to a day of light. So every time he pays that amount, he has a day of light. After 24 hours the system hangs and it has to make another payment to take advantage of the light. He can decide to pay for more than one day of light. The day number is then the whole part of the amount / daily rate ratio. After this number of days, the system still locks. When a customer spends more than 30 days without light, he is considered inactive. Paid vs Expected (PvE) means the ratio between the total amount paid by the client and the amount he should have paid up to date. It allows to appreciate the recovery.

Let's consider the following tables

Loan_current_daitails

loan_id	|customer_id|daily_rate|days_elapsed|Days_til_lock_out|Total_paid
--------|-----------|----------|------------|-----------------|----------
loan_id_1|customer_id_1|daily_rate_1|days_elapsed_1|Days_til_lock_out_1|Total_paid_1
loan_id_2|customer_id_2|daily_rate_2|days_elapsed_2|Days_til_lock_out_2|Total_paid_2
.........|.............|............|..............|...................|............
loan_id_n|customer_id_n	|daily_rate_n|days_elapsed_n|Days_til_lock_out_n|Total_paid_n


Person_democraphic

customer_id	|country|	region|	profession
------------|-------|---------|-----------
customer_id_1|	Country_1	|Region_1	|profession_1
.............|	............|..........	|............
customer_id_j|	Country_j|	Region_j	|profession_j


Sales_details

loan_id	|Product_type	|Date_fullfiled	|loan_duration
--------|----------------|--------------|-------------
loan_id_1	|Product_type_1	|Date_fullfiled_1	|loan_duration_1
...|	...	|...	|...
loan_id_n|	Product_type_n|	Date_fullfiled_n|	loan_duration_n

### Tables description

* **Loan_id:** unique loan identifier

* **Customer_id:** unique customer identifier

* **Days_elapsed:** the number of days the client spends with us

* **Daily_rate:** expected amount from customer per day

* **Days_til_lock_out:** If positive, then corresponds to the number of days of light remaining before the system hangs. If negative, corresponds to the number of days since the system was blocked (the client has been without light since this number of days)

* **Total_paid:** total amount to be paid by the customer

* **Country:** Customer's country

* **Region:** customer department

* **Profession:** client's profession

* **Product_type:** the type of kit purchased by the customer

* **Loan_duration:** the duration of the customer's loan (360 or 720 days)

* **Date_fullfiled:** the date of the sale

### The datasets
The datasets exist in csv formats and can be downloaded and saved on disk

In [52]:
%load_ext sql
%sql sqlite:///testdb.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @testdb.sqlite'

In [53]:
%reload_ext sql

In [54]:
%%sql
DROP TABLE IF EXISTS Loan_current_daitails;
DROP TABLE IF EXISTS Person_democraphic;
DROP TABLE IF EXISTS Sales_details;

 * sqlite:///testdb.sqlite
Done.
Done.
Done.


[]

In [55]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("testdb.sqlite")
cur = conn.cursor()
# print(conn.)
customers = pd.read_csv('./data/Loan_current_details.csv')
orders = pd.read_csv('./data/Person_democraphic.csv')
salesman = pd.read_csv('./data/Sales_details.csv')

In [57]:
cur.execute('DROP TABLE IF EXISTS Loan_current_daitails')
cur.execute('DROP TABLE IF EXISTS Person_democraphic')
cur.execute('DROP TABLE IF EXISTS Sales_details')

In [58]:
customers.to_sql("Loan_current_details", conn, index=False, if_exists="replace")
orders.to_sql("Person_democraphic", conn, index=False, if_exists="replace")
salesman.to_sql("Sales_details", conn, index=False, if_exists="replace")

/home/patrick/.local/lib/python3.6/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


> View SQL Tables

In [60]:
%%sql
SELECT * FROM Sales_details LIMIT 1

 * sqlite:///testdb.sqlite
Done.


loan_id,product_type,date_fulfilled_utc,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
589556,Fenix 3,6/5/19 10:45,None,None,None,None


In [61]:
%%sql
SELECT * FROM Person_democraphic LIMIT 1

 * sqlite:///testdb.sqlite
Done.


customer_id,country,region,Profession,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
1351176,Benin,Oueme,Tehnician,None,None,None,None


In [62]:
%%sql
SELECT * FROM Loan_current_details LIMIT 1

 * sqlite:///testdb.sqlite
Done.


loan_id,customer_id,daily_rate,days_elapsed,days_til_lockout,total_paid
589556,1351176,270,265,-13,52355


### Write a SQL code to determine the total number of loans on the one hand and the total number of customers on the other

In [9]:
%%sql
SELECT COUNT(loan_id) FROM Loan_current_details

 * sqlite:///testdb.sqlite
Done.


COUNT(loan_id)
5000


> `5000 Loans` have been underwritten

In [66]:
%%sql
SELECT COUNT(DISTINCT(customer_id)) FROM Person_democraphic

 * sqlite:///testdb.sqlite
Done.


COUNT(DISTINCT(customer_id))
4979


> There are `4979 customers`

### Write a SQL code to determine the total number of loans by type of kit

In [68]:
%%sql
SELECT  product_type kit_type, COUNT(DISTINCT(loan_id)) loan_count
FROM Sales_details
GROUP BY product_type

 * sqlite:///testdb.sqlite
Done.


kit_type,loan_count
Fenix 2,841
Fenix 3,974
Fenix Power 2+ Tondeuse,18
Fenix Power 4 + Woofer BJ,35
Fenix Power TV 19in FTA BJ,14
"Fenix Power TV 19in FTA Upgrade, BJ",28
Fenix Power TV 24in FTA BJ,634
"Fenix Power TV 24in FTA Upgrade, BJ",177
Fenix Radio +3,1791
Fenix Radio +5,422


### Write SQL code to determine the number of inactive loans

In [69]:
%%sql
SELECT COUNT(days_til_lockout) no_of_inactive_loans FROM Loan_current_details
WHERE days_til_lockout < 0

 * sqlite:///testdb.sqlite
Done.


no_of_inactive_loans
1294


### Write a SQL code to determine the regions where the total amount of payments exceeds 1,000,000

In [74]:
%%sql
SELECT pd.region, SUM(lcd.total_paid) amounts
FROM Person_democraphic pd
JOIN Loan_current_details lcd 
    ON  lcd.customer_id =pd.customer_id
GROUP BY region
HAVING amounts > 1000000

 * sqlite:///testdb.sqlite
Done.


region,amounts
Alibori,8559985
Atacora,7802685
Atlantique,51293780
Borgou,18947292
Collines,11904427
Couffo,4812070
Donga,9936982
Littoral,3980519
Mono,11621621
Oueme,30027194


### Write a SQL code to classify by region and in ascending order, the loans according to the total amount paid

In [75]:
%%sql
SELECT pd.region, SUM(lcd.total_paid) loans
FROM Person_democraphic pd
JOIN Loan_current_details lcd ON  lcd.customer_id =pd.customer_id
GROUP BY pd.region
ORDER BY pd.region ASC

 * sqlite:///testdb.sqlite
Done.


region,loans
Alibori,8559985
Atacora,7802685
Atlantique,51293780
Borgou,18947292
Collines,11904427
Couffo,4812070
Donga,9936982
Littoral,3980519
Mono,11621621
Oueme,30027194



### Write a SQL code to determine the top 10 occupations by region with the highest total amount of payments

In [80]:
%%sql
SELECT pd.region, pd.'Profession ' AS occupation, lcd.total_paid AS amounts
FROM Person_democraphic pd
JOIN Loan_current_details lcd ON  lcd.customer_id = pd.customer_id
GROUP BY pd.region, lcd.total_paid
ORDER BY lcd.total_paid DESC
LIMIT 10

 * sqlite:///testdb.sqlite
Done.


region,occupation,amounts
Zou,Cmmercial,287850
Atacora,Nurse,280000
Atlantique,Teacher,280000
Littoral,Nurse,280000
Donga,Statistian,252900
Borgou,Farmer,220000
Couffo,Nurse,212000
Plateau,Tehnician,212000
Atacora,Cmmercial,209300
Atlantique,Security agent,194500
